# Exercise sheet 3
### Due 18/11/2022, 14:00 

### Group:
-  
-  
-  

# The concept

A popular integration technique is the **trapezoid** method, which can be implemented to different degrees of accuracy $n$.

To first degree ($n=0$), the trapezoid method is just the area under a trapezoid connecting the two end points of our function:

$
\begin{align}
n=0: \int_a^b f(x) dx \,\approx\, I_0 = \frac{1}{2} h_0 [f(a) + f(b)], \hspace{5mm}\mathrm{where}\;h_0 = (b - a) \hspace{5cm}(1)
\end{align}
$

$I_0$ gives a very crude estimate of the integral. To increase our precision, we can divide the region between $a$ and $b$ into $n+1$ segments, calculate the area within these segments, and add them up. The formula for the n-th degree of accuracy of this algorithm is:

$
\begin{align}
n>0: \int_a^b f(x) dx \,\approx\, I_n = \frac{1}{2} I_{n-1} + h_n\sum_{i=1,3.,,,}^{2^n-1} f(a + i h_n), \hspace{5mm}\mathrm{where}\;h_n=\frac{1}{2}h_{n-1} \hspace{3.3cm}(2)
\end{align}
$

As you can see, with this algorithm the $n$-th degree approximation, $I_n$, is given as a recurrence relation, i.e. $I_n=I_n(I_{n-1})$.

As you calculate the algorithm for higher degree $n$, your approximation $I_n$ assimptotically approaches the result of the integral. The precision of your approximation can then be estimated by looking at the relative change between the last two steps:

$
\begin{align}
\varepsilon(n) = \frac{I_{n} - I_{n-1}}{I_n} \hspace{10cm}(3)
\end{align}
$

# The complex-looking task

You'll be building a function `trapz(f,a,b, prec)` that calculates the definite integral of a function `f` between `a` and `b`, down to a precision of `prec`.

For this, `trapz` calculates approximations of the integral using the trapezoidal algorithm, starting from $n=0$ and successively increasing $n$ until $|\varepsilon(n)|<$`prec` and our precision is satisfied. 

Once the precision is satisfied at $n=n_\mathrm{final}$, our function returns $I_{n_\mathrm{final}}$, $n_\mathrm{final}$, and $\varepsilon(n_\mathrm{final})$.

# The step-by-step breakdown

#### 1. Write a function `h_n` that returns the value of $h_n(a,b)$.

- Tip 1: Check out the example of a recurrence function we worked out in the lecture. 
- Tip 2: Remember that $h_n$ is defined recursively for $n>0$ (eq. 2) and has a special definition for $n=0$ (see eq. 1)

In [3]:
def h_n(n,a,b):
    '''Recursively calculate the term h_n(a,b).'''
    resh = (b-a)
    if n > 0 :
        resh = 0.5 * h_n(n-1,a,b)
    
    return resh

In [4]:
h_n(3,1,3)

4 * h_n(2,1,3)

2.0

#### 2. Write a function `sum_terms` that calculates the sum in eq. (2).

- Tip 1: for a given `n` (passed to `sum_terms` as an argument), you must iterate over `i` from 1 to $2^n-1$ in steps of 2
- Tip 2: remember that this sum depends on the function `f` that the user ultimately wants to integrate, so it has to take it as an argument additionally to `n`, `a` and `b`

In [30]:
def sum_terms(f,n,a,b):
    resA = 0
    for i in range(1, 2**n, 2):
        #print("i", i)
        resA = resA + f(a + i * h_n(n,a,b) )
        #print("resA", resA)
    #if i < 2**n :
        #resA = resA + f( a + i * h_n(n,a,b) )
        #resA = resA + sum_terms(f,n,a,b,i+2)
        #print(resA)
        #print(f"{i} und {resA}")
    
    return resA

In [31]:
sum_terms(lambda x: x ,4,1,4)

20.0

#### 3. Write a function `approx_n` that calculates the value of $I_n(a,b)$ for any given function `f`

- Tip 1: make use of `h_n` and `sum_terms` and remember to pass on the arguments
- Tip 2: remember that like $h_n$, $I_n$ is also defined recursively for $n>0$ (eq.1) and has a special definition for the initial estimate $I_0$ (eq. 1)

In [32]:
def approx_n(f,n,a,b):
    if n == 0:
        return 0.5 * h_n(0,a,b) * ( f(a) + f(b) )
    if n > 0:
        resI = 0.5 * approx_n(f,n-1,a,b) + h_n(n,a,b) * sum_terms(f,n,a,b)
    return resI

In [53]:
approx_n(lambda x:  x, 12,1,5)
approx_n(f2, 1,-2,2)

0.0

#### 4. Write a function `absepsilon` that takes two different approxiation values and returns the value of $|\varepsilon|$ between them (eq. 3)

- Tip: you can use Python's native abs() function 

In [54]:
def absepsilon(f,n,a,b):
    if approx_n(f,n,a,b) == 0:
        epsilon = 0
    else:
        epsilon = (abs(approx_n(f,n,a,b) - approx_n(f,n-1,a,b) )) / approx_n(f,n,a,b)
    
    return epsilon

In [55]:
absepsilon(lambda x: x ** 3, 5, -1,2)

0.005264184051472022

#### 5. Finally, put all the above functions together to create your integrator in the form of a function `trapz(f,a,b, prec)`. Make `prec` a keyword argument so that a precision of `1e-5` is assumed if the user doesn't specify any value. Once the precision is met, the function should return $I_n$, $n$ and $\varepsilon$.

- Tip 1: first, calculate $I_0$. Then starting with $n=1$, calculate a new approximation $I_n$ and compare it with the latest approximation. While your $\varepsilon$ is larger than the desired precision `prec`, keep increasing $n$, replacing the old approximation with the latest one.  

- Tip 2: make your life easier by printing the successive values of $I_n$, $n$ and $\varepsilon$ in every iteration, so you can see what is happening 

In [67]:
def trapz(f,a,b, prec = 1e-5):
    #I_0 = approx_n(f,0,a,b)
    
    n = 1
    
    while absepsilon(f,n,a,b) > prec:
        print(absepsilon(f,n,a,b))
        print(n)
        n += 1
    
    I_n = approx_n(f,n,a,b)
    epsilon = absepsilon(f,n,a,b)
    
    return I_n, n, epsilon

#### 6. Test your function by computing some integrals:

$
\begin{align}
\int_{-1}^{2} x^3 dx = \left[ \frac{x^4}{4} \right]_{-1}^{2} = 3.75
\end{align}
$


$
\begin{align}
\int_{-a}^{a} (x^5 + 2x^3) dx = 0 \hspace{5mm} \forall a \in \mathbb{R} \hspace{5mm}\text{(try $a=10^{-5}$ and $a=10^5$)}
\end{align}
$


In [68]:
f1 = lambda x: x ** 3
f2 = lambda x: x ** 5 + 2 * x ** 3
print(trapz(f1, -1,2))
print(trapz(f2, -1e-5,1e-5))
print(trapz(f2, -1e5,1e5))

0.9310344827586207
1
0.30337078651685395
2
0.08206686930091185
3
0.02094647013188518
4
0.005264184051472022
5
0.0013177802723412563
6
0.00032955363790599176
7
8.239519788580026e-05
8
2.0599223790730197e-05
9
(3.7500064373016357, 10, 5.149832468320424e-06)
(0.0, 1, 0)
(0.0, 1, 0)


## Bonus exercise!

An *extra* 3 points are up for grabs if you optimize your algorithm so that every time you calculate $I_{n}$ you don't to calculate $I_{n-1}$ iteratively  (perhaps by passing the latest value of $I_{n-1}$ to your `approx_n` function as a keyword argument...)

This will greatly increase the performance of your function!


### Notes

- Remember to document all your functions with a docstring. For helper funcitons a one-liner may be enough, but for `trapz` a proper description is necessary (see notebook from lecture #3)

- There are Python libraries with tools to integrate functions through many different methods, and we're going to be working with them later on in the course. But for this task you should use **only native Python tools**, because the goal is to become comfortable creating and manipulating functions in Python.